In [ ]:
#@title Bước 1
#@markdown Chạy cell này xong sẽ bị báo:
#@markdown >Your session crashed for an unknown reason.

#@markdown Bỏ qua thông báo này và tiếp tục chạy cell ở bước 2.
!add-apt-repository -y ppa:sergey-dryabzhinsky/packages
!apt update
!apt install sqlite3

import os
os.kill(os.getpid(), 9)

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net

In [1]:
#@title Bước 2
#@markdown Bạn hãy kiểm tra xem kết quả in ra của cell này phải lớn hơn `3.38.0`.

#@markdown Nếu không đúng (vd. `3.22.0`) thì hãy kết thúc runtime, tạo session mới và chạy lại 2 cell này theo đúng thứ tự.

!pip install -U --no-cache-dir gdown >/dev/null 2>&1
!gdown -q 1nEdM0RLbs_lT4ibG0YLc6jmrM1F9DPh2 -O ./E_commerce

import sqlite3
print(sqlite3.sqlite_version)
import pandas as pd
conn = sqlite3.connect('E_commerce')

3.39.4


![](https://storage.googleapis.com/kaggle-datasets-images/55151/105464/d59245a7014a35a35cc7f7b721de4dae/dataset-cover.png?t=2018-09-21-16-21-21)

This is a dataset on BigQuery about __Brazillian E-commerce__ (Source: [Kaggle](https://www.kaggle.com/olistbr/brazilian-ecommerce)). 


![](https://drive.google.com/uc?export=view&id=1nuLGF7VnRJmMZq7BkS1hPFLOZTxGd-UA)

In [2]:
# show all the table names
query = '''
SELECT name
FROM sqlite_master 
    WHERE type ='table' 
      AND name NOT LIKE 'sqlite_%'
'''

pd.read_sql_query(query, conn)

,name
0,customers
1,sellers
2,products
3,order_reviews
4,order_payments
5,order_items
6,catergory_name_translation
7,geo_location
8,orders


## Questions:


> 1/ Top 10 categories that have the highest number of products


> 2/ Top 5 seller_id by the total value of orders delivered to customers


> 3/ Find top 5 best-selling product categories


> 4/ Statistic the revenue of all sellers by month


> 5/ Statistics the seller's information according to given table


### Question 1: Top 10 categories that have the highest number of products

In [46]:
query = '''
WITH tmp as (SELECT product_category_name, count(*) as Product_count
FROM products as p
GROUP BY product_category_name
ORDER BY product_count desc
LIMIT 10),

category as (SELECT distinct(category_name) as cat, category_name_english
FROM catergory_name_translation)

SELECT category_name_english, product_count
FROM tmp LEFT JOIN category on tmp.product_category_name = category.cat
'''

pd.read_sql_query(query, conn)

,category_name_english,Product_count
0,bed_bath_table,3029
1,sports_leisure,2867
2,furniture_decor,2657
3,health_beauty,2444
4,housewares,2335
5,auto,1900
6,computers_accessories,1639
7,toys,1411
8,watches_gifts,1329
9,telephony,1134


### Question 2: Top 5 seller_id by the total value of orders delivered to customers

In [47]:
query = '''
WITH data as (
  SELECT * 
  FROM order_items as i
    JOIN orders as o on i.order_id = o.order_id
  WHERE o.order_status = 'delivered' AND order_delivered_customer_date is not null
  ),
  
data1 as (
  SELECT *, sum(price + freight_value) as value
  FROM data
  GROUP BY order_id, order_item_id)
  
SELECT seller_id, sum(value) as total
FROM data1
GROUP BY seller_id
ORDER BY total desc
LIMIT 5
  '''

pd.read_sql_query(query, conn)

,seller_id,total
0,4869f7a5dfa277a7dca6462dcf3b52b2,247007.06
1,7c67e1448b00f6e969d365cea6b010ab,237806.69
2,4a3ca9315b744ce9f8e9374361493884,231220.43
3,53243585a1d6dc2643021fd1853d8905,230797.02
4,fa1c13f2614d7b5c4749cbc52fecda94,200833.50


### Question 3: Find top 5 best-selling product categories

In [3]:
query = '''
WITH tmp as (
  SELECT *
  FROM order_items i 
  LEFT JOIN products p on i.product_id = p.product_id
  LEFT JOIN catergory_name_translation C on p.product_category_name = c.category_name)


SELECT category_name_english, count(order_id) as number_sold
FROM tmp
GROUP BY category_name_english
ORDER BY number_sold desc
LIMIT 5
'''

pd.read_sql_query(query, conn)

,category_name_english,number_sold
0,bed_bath_table,11115
1,health_beauty,9670
2,sports_leisure,8641
3,furniture_decor,8334
4,computers_accessories,7827


### Question 4: Statistic the revenue of all sellers by month. 

In [50]:
query = '''
WITH tmp as (
  SELECT *, strftime('%m', order_approved_at) AS month, strftime('%Y', order_approved_at) AS year
  FROM order_items i 
  JOIN sellers s on i.seller_id = s.seller_id
  JOIN orders o on o.order_id = i.order_id
  WHERE order_approved_at != ''
)

SELECT (month || "/" || year) as month, revenue
FROM (
  SELECT year, month, sum(price + freight_value) as revenue
  FROM tmp
  GROUP BY year, month
  ORDER BY year, month
)
'''


pd.read_sql_query(query, conn)

,month,revenue
0,09/2016,143.46
1,10/2016,57020.13
2,12/2016,19.62
3,01/2017,130840.55
4,02/2017,286135.92
5,03/2017,428557.47
6,04/2017,409570.85
7,05/2017,587151.63
8,06/2017,507643.19
9,07/2017,578682.67


### Question 5: Statistics the seller's information according to the table below. 

Where:

```
number_of_product: the total of distinct products were sold (not the amount of product was sold)
product_category_revenue: the revenue of a product category of a seller
rank_in_category: rank by revenue of a product category of a seller 
total_revenue: the total revenue of a seller
```

In [45]:
query = '''
WITH tmp as (
  SELECT *
  FROM order_items i 
    JOIN sellers s on i.seller_id = s.seller_id
    JOIN orders o on o.order_id = i.order_id
    JOIN products p on p.product_id = i.product_id
    JOIN catergory_name_translation C on p.product_category_name = c.category_name
    WHERE order_approved_at != ''),
    
tmp2 as (
  SELECT seller_id, sum(price + freight_value) as total_revenue, count(distinct(product_id)) as number_of_products
  FROM tmp
  GROUP BY seller_id
  ORDER BY total_revenue desc),

tmp3 as (
  SELECT seller_id, category_name_english as category_name, sum(price + freight_value) as product_category_revenue
  FROM tmp
  GROUP BY seller_id, category_name
  ORDER BY category_name, product_category_revenue desc),
  
tmp4 as (
  SELECT *, RANK() OVER(PARTITION BY category_name ORDER BY product_category_revenue desc) rank_in_category
  FROM tmp3)

SELECT *
FROM tmp4 LEFT JOIN tmp2 USING(seller_id)
'''


pd.read_sql_query(query, conn)

,seller_id,category_name,product_category_revenue,rank_in_category,total_revenue,number_of_products
0,e59aa562b9f8076dd550fcddf0e73491,agro_industry_and_commerce,33803.97,1,33803.97,9
1,6bd69102ab48df500790a8cecfc285c2,agro_industry_and_commerce,8359.02,2,8359.02,3
2,f08a5b9dd6767129688d001acafc21e5,agro_industry_and_commerce,7796.21,3,14565.74,7
3,2528744c5ef5d955adc318720a94d2e7,agro_industry_and_commerce,6508.70,4,6508.70,3
4,31ae0774c17fabd06ff707cc5bde005f,agro_industry_and_commerce,5690.23,5,7570.67,9
...,...,...,...,...,...,...
6346,2ff97219cb8622eaf3cd89b7d9c09824,watches_gifts,56.22,97,5866.03,45
6347,891071be6ba827b591264c90c2ae8a63,watches_gifts,35.96,98,5454.35,31
6348,b98b715fe9e276204e2a3464f48cd796,watches_gifts,33.71,99,517.62,4
6349,1da3aeb70d7989d1e6d9b0e887f97c23,watches_gifts,27.22,100,13055.36,155
